# Group-Specific Discriminant Analysis for sex-specific lateralization Running Demo

[Open in Colab](https://colab.research.google.com/github/shuo-zhou/GSDA-Lateralization/blob/main/gsda_demo.ipynb)  (click `Runtime` → `Run all (Ctrl+F9)`)

Online demo for the article ["Group-specific discriminant analysis reveals statistically validated sex differences in lateralization of brain functional network"](https://arxiv.org/abs/2404.05781)

## Setup

The first few blocks of code are necessary to set up the notebook execution environment. This checks if the notebook is running on Google Colab and installs required packages.

In [ ]:
#@title Setup environment and fetch code from GitHub

if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip install yacs
    !git clone https://github.com/shuo-zhou/GSDA-Lateralization
    %cd GSDA-Lateralization
else:
    print('Not running on CoLab')
     

In [ ]:
#@title Import required modules
import os
from configs.default_cfg import get_cfg_defaults
from utils.experiment import run_experiment
from joblib import Parallel, delayed

from utils.io_ import load_result, reformat_results
from utils import plot

### Configurations

The customized configuration used in this demo is stored in `configs/demoh-cp.yaml`, this file overwrites defaults in `default_cfg.py` where a value is specified. Change the configuration file path to `cfg_path = "configs/demo-gsp.yaml"` for running the demo with GSP data. The processed intrahemispheric functional connectivity data is publicly available on Zenodo at

- [Human Connectome Project (HCP)](https://zenodo.org/records/10050233)
- [Brain Genomics Superstruct Project (GSP)](https://zenodo.org/records/10050234)

In [ ]:
#@title Setup configs using .yaml file
cfg_path = "configs/demo-hcp.yaml" # Path to `.yaml` config file

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
print(cfg)

## Group-specific model training

It could take a while (15 to 25 mins) to run the experiments. 

In [ ]:
_ = Parallel(n_jobs=2)(delayed(run_experiment)(cfg, lambda_) for lambda_ in cfg.SOLVER.LAMBDA_)

# print() doesn't work with `Parallel` on Colab, please comment out the above line and uncomment the following line 
# if you are would like to see the outputs during training. Please be aware that this would slow down the whole training process.
# _ = [run_experiment(cfg, lambda_) for lambda_ in cfg.SOLVER.LAMBDA_]

## Classification result visualization

In [ ]:
#@title Load results and reformat as a dataframe

dataset = cfg.DATASET.DATASET
model_root_dir = cfg.OUTPUT.ROOT
lambdas = cfg.SOLVER.LAMBDA_
seed_start = cfg.SOLVER.SEED
test_size = cfg.DATASET.TEST_RATIO

res_df = load_result(dataset=dataset, root_dir=model_root_dir, 
                         lambdas=lambdas, seed_start=seed_start, test_size=test_size)

res_df["GSI_train_session"] = 2 * (res_df["acc_tgt_train_session"] * 
                                       (res_df["acc_tgt_train_session"] - 
                                        res_df["acc_nt_train_session"]))
res_df["GSI_test_session"] = 2 * (res_df["acc_tgt_test_session"] * 
                                      (res_df["acc_tgt_test_session"] - 
                                       res_df["acc_nt_test_session"]))

res_df_train_session = reformat_results(res_df, ["acc_tgt_train_session", "acc_nt_train_session"])
res_df_test_session = reformat_results(res_df, ["acc_tgt_test_session", "acc_nt_test_session"])

In [ ]:
#@title Plot accuracy

if not os.path.exists("figures"):
    os.mkdir("figures")
plot.plot_accuracy(res_df_train_session)

In [ ]:
#@title Plot Group Specificity Index (GSI)
plot.plot_gsi(res_df, x="lambda", y="GSI_train_session", hue="target_group")

## Citation

If you find this work useful, please cite the following paper:

```(bibtex)
@misc{zhou2024groupspecific,
      title={Group-specific discriminant analysis reveals statistically validated sex differences in lateralization of brain functional network},
      author={Shuo Zhou and Junhao Luo and Yaya Jiang and Haolin Wang and Haiping Lu and Gaolang Gong},
      year={2024},
      eprint={2404.05781},
      archivePrefix={arXiv},
      primaryClass={q-bio.NC}
}